<a href="https://colab.research.google.com/github/FranciscoFoz/validacao-cadastro-usuarios-biblioteca/blob/main/Notebook/Validacao_cadastro_usuario_biblioteca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Validação de cadastro de usuário biblioteca

## Resumo

Uma biblioteca universitária, no Brasil, está validando os dados do cadastro de seus usuários, para que possam entrar em contato. O e-mail é a comunicação ideal para entrar em contato, por isso já foi feita a validação. Entretanto nem todos os usuários possuem no cadastro um e-mail válido e a segunda opção seria o número de celular. Mas o número de celular não é validado inteiramente no formulário da página web do cadastro e números repetidos ou incorretos podem aparecer.

Quais são os usuários da biblioteca que não possuem um número de celular válido? Com essas informações a equipe da direção da biblioteca poderá montar um plano de ação para de alguma forma corrigir o cadastro.

Também está disponível a tabela destes usuários no banco de dados da matrícula da universidade e do cadastro inicial do comercial. Para verificar se não estão corretos nestes outros.

## Importação das bibliotecas

In [1]:
import re
import pandas as pd

## Importação da classe de validação

In [81]:
!pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=54a837d5a2804717fd68cb8c285f2f2d8829b8b2a6c9dc466df18465d050f554
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [ ]:
CelularBr = wget.download()

In [2]:
class CelularBr:
    """
    Esta classe irá validar o número de celular, retornando-o
    no formato padrão do Brasil: (xx) xxxxx-xxxx.

    Caso o número não seja válido, retornará a mensagem:
    "Número inválido"
    """
    def __init__(self, celular):
        self.celular = str(celular)

    def verifica_validacao(self):
        """
        Esta função irá verificar a validação do número retornado,
        pela função valida_celular.
        Se for válido, ela formatará nopadrão da formata_celular,
        se não for imprimirá "Número inválido"
        """
        if self.valida_celular(self.celular):
            return self.formata_numero()
        else:
            return "Número inválido"

    def valida_celular(self, celular):
        """
        Esta função irá validar (com Regex) o número de celular
        com o DDD.
        Retornando True ou False.
        """
        padrao = '[1-9]{2}9[5-9]{1}[0-9]{7}'
        primeira_validacao = re.findall(padrao, celular)
        if primeira_validacao:
            segundo_padrao = '0{6}|1{6}|2{6}|3{6}|4{6}|5{6}|6{6}|7{6}|8{6}|9{6}'
            segunda_validacao = re.findall(segundo_padrao,
                                           primeira_validacao[0])
            if segunda_validacao:
                return False
            else:
                return True
        else:
            return False

    def formata_numero(self):
        """
        Esta função retornará o o número formatado no padrão:
        (xx) xxxxx-xxxx.
        """
        padrao = '([1-9]{2})(9[0-9]{4})([0-9]{4})'
        resposta = re.search(padrao, self.celular)

        ddd = resposta.group(1)
        primeiros_digitos = resposta.group(2)
        ultimos_digitos = resposta.group(3)
        numero_formatado = (f"({ddd}) {primeiros_digitos}-{ultimos_digitos}")

        return numero_formatado

In [3]:
biblioteca = pd.read_csv('/content/biblioteca.csv')
comercial = pd.read_csv('/content/comercial.csv')
matricula = pd.read_csv('/content/matricula.csv')

In [4]:
biblioteca

,ID,RA,TELEFONE,EMAIL
0,38,202210435,68976666666,Inválido
1,46,202210513,68991128727,Inválido
2,52,202210551,68980743830,Inválido
3,53,202210574,68982847367,Inválido
4,80,202210828,68996455542,Inválido
...,...,...,...,...
995,771,202217410,63979328523,Inválido
996,804,202217742,63979489768,Inválido
997,809,202217760,63988888888,Inválido
998,840,202218170,63995130349,Inválido


In [7]:
biblioteca['TELEFONE_VALIDADO'] = biblioteca['TELEFONE'].apply(lambda celular: CelularBr(celular).verifica_validacao())

In [8]:
biblioteca

,ID,RA,TELEFONE,EMAIL,TELEFONE_VALIDADO
0,38,202210435,68976666666,Inválido,Número inválido
1,46,202210513,68991128727,Inválido,(68) 99112-8727
2,52,202210551,68980743830,Inválido,(68) 98074-3830
3,53,202210574,68982847367,Inválido,(68) 98284-7367
4,80,202210828,68996455542,Inválido,(68) 99645-5542
...,...,...,...,...,...
995,771,202217410,63979328523,Inválido,(63) 97932-8523
996,804,202217742,63979489768,Inválido,(63) 97948-9768
997,809,202217760,63988888888,Inválido,Número inválido
998,840,202218170,63995130349,Inválido,(63) 99513-0349


In [18]:
numeros_invalidos = biblioteca.query('TELEFONE_VALIDADO == "Número inválido"')
numeros_invalidos = numeros_invalidos.sort_values(by='ID',ascending=True)
numeros_invalidos

,ID,RA,TELEFONE,EMAIL,TELEFONE_VALIDADO
0,38,202210435,68976666666,Inválido,Número inválido
256,60,202210654,27999999999,Inválido,Número inválido
257,65,202210685,27966666666,Inválido,Número inválido
693,79,202210827,11111111111,Inválido,Número inválido
696,125,202211334,84888888888,Inválido,Número inválido
964,209,202212145,63999999999,Inválido,Número inválido
705,242,202212419,84922222222,Inválido,Número inválido
854,252,202212509,48999999999,Inválido,Número inválido
855,292,202212962,48911111111,Inválido,Número inválido
664,331,202213278,21212121212,Inválido,Número inválido


In [19]:
len(numeros_invalidos)

30

In [20]:
matricula

,ID,RA,UF,CIDADE,TELEFONE,EMAIL,CURSO
0,38,202210435,AC,Rio Branco,6.897667e+10,Inválido,Negócios Imobiliários
1,46,202210513,AC,Rio Branco,6.899113e+10,Inválido,Negócios Imobiliários
2,52,202210551,AC,Rio Branco,6.898074e+10,Inválido,Educação Física
3,53,202210574,AC,Rio Branco,6.898285e+10,Inválido,Ciência de Dados
4,80,202210828,AC,Rio Branco,6.899646e+10,Inválido,Arquitetura de Dados
...,...,...,...,...,...,...,...
1004,771,202217410,TO,Palmas,6.397933e+10,Inválido,Produção Cervejeira
1005,804,202217742,TO,Palmas,6.397949e+10,Inválido,Teologia
1006,809,202217760,TO,Palmas,6.398889e+10,Inválido,Radiologia
1007,840,202218170,TO,Palmas,6.399513e+10,Inválido,Filosofia


In [21]:
id_numeros_invalidos = list(numeros_invalidos.ID)

In [30]:
matricula_telefone = matricula.query('ID == @id_numeros_invalidos')[['ID','TELEFONE']]
matricula_telefone

,ID,TELEFONE
0,38,6.897667e+10
34,991,6.887152e+10
71,673,8.299925e+10
104,565,9.297778e+10
118,894,NaN
262,60,2.800000e+10
263,65,2.796667e+10
670,331,2.121212e+10
671,400,1.010101e+10
699,79,1.111111e+10


In [31]:
matricula_telefone = matricula_telefone.sort_values(by='ID',ascending=True)
matricula_telefone = matricula_telefone.dropna()
matricula_telefone = matricula_telefone.astype(int)
matricula_telefone['TELEFONE_VALIDADO'] = matricula_telefone['TELEFONE'].apply(lambda celular: CelularBr(celular).verifica_validacao())
matricula_telefone

,ID,TELEFONE,TELEFONE_VALIDADO
0,38,68976666666,Número inválido
262,60,27999999999,Número inválido
263,65,27966666666,Número inválido
699,79,11111111111,Número inválido
702,125,84888888888,Número inválido
973,209,63999999999,Número inválido
711,242,84922222222,Número inválido
863,252,48999999999,Número inválido
864,292,48911111111,Número inválido
670,331,21212121212,Número inválido


In [34]:
matricula_telefone_validado = matricula_telefone.query('TELEFONE_VALIDADO != "Número inválido"')
matricula_telefone_validado

,ID,TELEFONE,TELEFONE_VALIDADO
992,490,63991250338,(63) 99125-0338
71,673,82999250449,(82) 99925-0449
915,684,79991450669,(79) 99145-0669


In [38]:
id_matricula_telefone_invalido = list(matricula_telefone.query('TELEFONE_VALIDADO == "Número inválido"').ID)
id_matricula_telefone_invalido

[38,
 60,
 65,
 79,
 125,
 209,
 242,
 252,
 292,
 331,
 400,
 414,
 473,
 520,
 552,
 565,
 592,
 632,
 635,
 755,
 769,
 809,
 869,
 901,
 979,
 991]

In [50]:
comercial_telefone = comercial.query('ID == @id_matricula_telefone_invalido')[['ID','TELEFONE']]
comercial_telefone


,ID,TELEFONE
0,38,6.897667e+10
1,38,6.897667e+10
2,38,6.897667e+10
40,991,6.887152e+10
110,565,9.297778e+10
268,60,2.800000e+10
269,65,2.796667e+10
676,331,2.121212e+10
677,400,1.010101e+10
705,79,1.111111e+10


In [51]:
comercial_telefone = comercial_telefone.astype(int)
comercial_telefone['TELEFONE_VALIDADO'] = comercial_telefone['TELEFONE'].apply(lambda celular: CelularBr(celular).verifica_validacao())
comercial_telefone

,ID,TELEFONE,TELEFONE_VALIDADO
0,38,68976666666,Número inválido
1,38,68976666666,Número inválido
2,38,68976666666,Número inválido
40,991,68871520623,Número inválido
110,565,92977777777,Número inválido
268,60,27999999999,Número inválido
269,65,27966666666,Número inválido
676,331,21212121212,Número inválido
677,400,10101010101,Número inválido
705,79,11111111111,Número inválido


matricula_telefone_validado = matricula_telefone.query('TELEFONE_VALIDADO != "Número inválido"')
matricula_telefone_validado

id_matricula_telefone_invalido = list(matricula_telefone.query('TELEFONE_VALIDADO == "Número inválido"').ID)
id_matricula_telefone_invalido

In [52]:
comercial_telefone_validado = comercial_telefone.query('TELEFONE_VALIDADO != "Número inválido"') 
comercial_telefone_validado

,ID,TELEFONE,TELEFONE_VALIDADO
730,520,84991450228,(84) 99145-0228
784,414,95991759942,(95) 99175-9942
918,592,79981450669,(79) 98145-0669
961,635,11991350889,(11) 99135-0889


In [53]:
id_comercial_telefone_invalido = list(comercial_telefone.query('TELEFONE_VALIDADO == "Número inválido"').ID) 
id_comercial_telefone_invalido

[38,
 38,
 38,
 991,
 565,
 60,
 65,
 331,
 400,
 79,
 125,
 242,
 632,
 473,
 552,
 252,
 292,
 769,
 979,
 755,
 869,
 901,
 209,
 809]

In [70]:
numeros_invalidos

,ID,RA,TELEFONE,EMAIL,TELEFONE_VALIDADO
0,38,202210435,68976666666,Inválido,Número inválido
256,60,202210654,27999999999,Inválido,Número inválido
257,65,202210685,27966666666,Inválido,Número inválido
693,79,202210827,11111111111,Inválido,Número inválido
696,125,202211334,84888888888,Inválido,Número inválido
964,209,202212145,63999999999,Inválido,Número inválido
705,242,202212419,84922222222,Inválido,Número inválido
854,252,202212509,48999999999,Inválido,Número inválido
855,292,202212962,48911111111,Inválido,Número inválido
664,331,202213278,21212121212,Inválido,Número inválido


In [75]:
numeros_invalidos

,ID,RA,TELEFONE,EMAIL,TELEFONE_VALIDADO
0,38,202210435,68976666666,Inválido,Número inválido
256,60,202210654,27999999999,Inválido,Número inválido
257,65,202210685,27966666666,Inválido,Número inválido
693,79,202210827,11111111111,Inválido,Número inválido
696,125,202211334,84888888888,Inválido,Número inválido
964,209,202212145,63999999999,Inválido,Número inválido
705,242,202212419,84922222222,Inválido,Número inválido
854,252,202212509,48999999999,Inválido,Número inválido
855,292,202212962,48911111111,Inválido,Número inválido
664,331,202213278,21212121212,Inválido,Número inválido


In [80]:
telefone_validados = pd.concat([comercial_telefone_validado,matricula_telefone_validado])
telefone_validados

,ID,TELEFONE,TELEFONE_VALIDADO
730,520,84991450228,(84) 99145-0228
784,414,95991759942,(95) 99175-9942
918,592,79981450669,(79) 98145-0669
961,635,11991350889,(11) 99135-0889
992,490,63991250338,(63) 99125-0338
71,673,82999250449,(82) 99925-0449
915,684,79991450669,(79) 99145-0669


In [55]:
total_numeros_invalidos = numeros_invalidos.query('ID == @id_comercial_telefone_invalido')
total_numeros_invalidos

,ID,RA,TELEFONE,EMAIL,TELEFONE_VALIDADO
0,38,202210435,68976666666,Inválido,Número inválido
256,60,202210654,27999999999,Inválido,Número inválido
257,65,202210685,27966666666,Inválido,Número inválido
693,79,202210827,11111111111,Inválido,Número inválido
696,125,202211334,84888888888,Inválido,Número inválido
964,209,202212145,63999999999,Inválido,Número inválido
705,242,202212419,84922222222,Inválido,Número inválido
854,252,202212509,48999999999,Inválido,Número inválido
855,292,202212962,48911111111,Inválido,Número inválido
664,331,202213278,21212121212,Inválido,Número inválido


In [68]:
total_numeros_invalidos = total_numeros_invalidos.merge(matricula[['ID','UF','CIDADE','CURSO']],how='left',left_on='ID', right_on='ID')
total_numeros_invalidos

,ID,RA,TELEFONE,EMAIL,TELEFONE_VALIDADO,UF,CIDADE,CURSO
0,38,202210435,68976666666,Inválido,Número inválido,AC,Rio Branco,Negócios Imobiliários
1,60,202210654,27999999999,Inválido,Número inválido,ES,Vitória,Nutrição
2,65,202210685,27966666666,Inválido,Número inválido,ES,Vitória,Design de Interiores
3,79,202210827,11111111111,Inválido,Número inválido,RN,Natal,Pedagogia
4,125,202211334,84888888888,Inválido,Número inválido,RN,Natal,Fotografia
5,209,202212145,63999999999,Inválido,Número inválido,TO,Palmas,Negócios Imobiliários
6,242,202212419,84922222222,Inválido,Número inválido,RN,Natal,Administração
7,252,202212509,48999999999,Inválido,Número inválido,SC,Florianópolis,Gestão Pública
8,292,202212962,48911111111,Inválido,Número inválido,SC,Florianópolis,Pedagogia
9,331,202213278,21212121212,Inválido,Número inválido,RJ,Rio de Janeiro,Engenharia Civil


In [69]:
len(total_numeros_invalidos)

22

In [ ]:
import timeit



In [ ]:
inicio = time.time()
[CelularBr(numero).verifica_validacao() for numero in biblioteca['TELEFONE']]
fim = time.time()
print(fim - inicio)


In [ ]:
inicio = time.time()
biblioteca['TELEFONE'].apply(lambda celular: CelularBr(celular).verifica_validacao())
fim = time.time()
print(fim - inicio)

In [ ]:
0.0429384708404541/0.013110876083374023
